Defining the autoencoder module

In [4]:
import os
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest


#r'C:\Users\gusta\Documents\KTH\TriOptima\trioptima/'
#'/Users/elliotlindestam/Documents/Skola/Indek icloud/trioptima/'
# Constants and Configurations
your_path = r'C:\Users\gusta\Documents\KTH\TriOptima\trioptima/'

def load_preprocessed_data(file_path):
    # Load preprocessed data
    data = pd.read_csv(file_path, header=0)
    # Convert data to float32
    data = data.astype(np.float32)
    # Check for inf or NaN values in data
    print("Inf values in data: ", np.isinf(data).values.sum())
    print("NaN values in data: ", np.isnan(data).values.sum())
    return data

def fit_isolation_forest(data):
    # Initialize Isolation Forest
    iso_forest = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
    # Fit the model
    iso_forest.fit(data)
    return iso_forest

def main():
    train_data_path = your_path + '6.Active Data/Train Model Data/Train_Scaled.csv'
    test_data_path = your_path + '6.Active Data/Test Data/Test_Scaled.csv'

    # Load preprocessed training and test data
    train_data_preprocessed = load_preprocessed_data(train_data_path)
    test_data_preprocessed = load_preprocessed_data(test_data_path)

    # Fit the Isolation Forest model
    iso_forest = fit_isolation_forest(train_data_preprocessed)

    # Predict anomalies on the test set
    test_predictions = iso_forest.predict(test_data_preprocessed)
    # Scores of the training data
    anomaly_scores = iso_forest.decision_function(test_data_preprocessed)

    # Anomalies are labeled as -1
    outlier_indices = np.where(test_predictions == -1)[0]

    # Output the number of anomalies detected
    print(f"Number of anomalies detected: {len(outlier_indices)}")

    # Assuming you want to maintain the same interface for mse calculation
    mse_test_df = pd.DataFrame({'reconstruction_error': -anomaly_scores})

    # Display basic statistics of the anomaly scores
    print(mse_test_df.describe())

    # Visualize the anomaly scores
    import matplotlib.pyplot as plt
    import seaborn as sns

    sns.histplot(mse_test_df['reconstruction_error'], bins=50, kde=True)
    plt.title('Anomaly score distribution')
    plt.xlabel('Anomaly score')
    plt.ylabel('Frequency')
    plt.show()

    return iso_forest, outlier_indices

if __name__ == "__main__":
    iso_forest, outlier_indices = main()

# Further code to process the outliers can be added below as per original logic, using `outlier_indices`


Inf values in data:  0
NaN values in data:  0
Epoch 1/10
51/51 [==============================] - 1s 6ms/step - loss: 84805.5625 - val_loss: 84767.6406
Epoch 2/10
51/51 [==============================] - 0s 3ms/step - loss: 84805.5234 - val_loss: 84767.6406
Epoch 3/10
51/51 [==============================] - 0s 3ms/step - loss: 84805.5156 - val_loss: 84767.6328
Epoch 4/10
51/51 [==============================] - 0s 4ms/step - loss: 84805.4844 - val_loss: 84767.6328
Epoch 5/10
51/51 [==============================] - 0s 5ms/step - loss: 84805.4922 - val_loss: 84767.6328
Epoch 6/10
51/51 [==============================] - 0s 4ms/step - loss: 84805.4844 - val_loss: 84767.6328
Epoch 7/10
51/51 [==============================] - 0s 4ms/step - loss: 84805.4922 - val_loss: 84767.6328
Epoch 8/10
51/51 [==============================] - 0s 5ms/step - loss: 84805.4609 - val_loss: 84767.6406
Epoch 9/10
51/51 [==============================] - 0s 5ms/step - loss: 84805.4609 - val_loss: 84767.6250
